In [1]:
seed = 0
import torch
torch.manual_seed(seed)
import random
random.seed(seed)
import numpy as np
np.random.seed(seed)
torch.use_deterministic_algorithms(True)

In [17]:
import torch
from torch import Tensor
from torch_geometric.logging import init_wandb, log
from torch_geometric.datasets import Planetoid
import torch.nn.functional as F
from utils import train, test, edgeindex2adj
citeseer = Planetoid(root='.', name='Citeseer')
cora = Planetoid(root='.', name='Cora')
# pubmed = Planetoid(root='.', name='Pubmed')

In [35]:
from models import ALP, GCN
dataset = cora
data = dataset[0]

# take k random training nodes for each class
k = 1
for c in data.y.unique():
    idx = ((data.y == c) & data.train_mask).nonzero(as_tuple=False).view(-1)
    idx = idx[torch.randperm(idx.size(0))]
    idx = idx[k:]
    data.train_mask[idx] = False

print(data.train_mask.sum())

tensor(7)


In [36]:
yshape = dataset[0].y.shape[0]
model = ALP(num_layers=16, alpha=0.9, yshape=yshape)
outs = model.lpa(dataset)

gcn = GCN(dataset.num_features, 8, dataset.num_classes)

optimizer_gcn = torch.optim.Adam(gcn.parameters(), lr=0.005, weight_decay=5e-4)
optimizer_alp = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

In [37]:
model.test()

[1.0, 0.316, 0.322]

In [38]:
train_mask = data.train_mask.clone()
val_mask = data.val_mask.clone()
test_mask = data.test_mask.clone()

unlab_mask = torch.ones_like(data.train_mask)
unlab_mask[(data.train_mask == True) | (data.val_mask == True) | (data.test_mask == True)] = False

print(train_mask.sum(), val_mask.sum(), test_mask.sum(), unlab_mask.sum())

tensor(7) tensor(500) tensor(1000) tensor(1201)


In [39]:
# for p in model.parameters():
#     print(p)

In [40]:
model.train()
best_val_acc = final_test_acc = 0
for epoch in range(1, 200):
    
    optimizer_gcn.zero_grad()
    optimizer_alp.zero_grad()

    # add 1% of unlabelled data to training data
    new_train_mask = train_mask.clone()
    to_add_mask = unlab_mask.clone()
    # put 99% to 0
    idx = to_add_mask.nonzero(as_tuple=False).view(-1)
    idx = idx[torch.randperm(idx.size(0))][:int(0.90 * idx.size(0))]
    to_add_mask[idx] = False
    new_train_mask[to_add_mask] = True
    # print(new_train_mask.sum())
    
    op_alp = model()
    op = gcn(data.x, data.edge_index)
    
    # print(op_alp.shape, op.shape, data.y[train_mask].shape)
    loss = F.cross_entropy(op[new_train_mask, :], op_alp[new_train_mask].argmax(dim=1))
    # print(op_alp[train_mask].shape, data.y[train_mask].shape)
    loss2 = F.cross_entropy(op_alp[train_mask], data.y[train_mask])
    # print(loss.item(), loss2.item())
    # for p in model.parameters():
    #     print(p)
    # loss += 0.01 * loss2
    loss.backward()
    
    optimizer_alp.step()
    optimizer_gcn.step()
    
    train_acc, val_acc, tmp_test_acc = test(gcn, data)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc, Loss2 = loss2)
print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 1.9535118341445923, Train: 0.1429, Val: 0.1480, Test: 0.1510, Loss2: 1.764222502708435
Epoch: 002, Loss: 1.9152154922485352, Train: 0.4286, Val: 0.1700, Test: 0.1580, Loss2: 1.764222502708435
Epoch: 003, Loss: 1.8757795095443726, Train: 0.4286, Val: 0.1820, Test: 0.1760, Loss2: 1.764222502708435
Epoch: 004, Loss: 1.8489302396774292, Train: 0.5714, Val: 0.1980, Test: 0.1820, Loss2: 1.764222502708435
Epoch: 005, Loss: 1.8029606342315674, Train: 0.5714, Val: 0.2060, Test: 0.1930, Loss2: 1.764222502708435
Epoch: 006, Loss: 1.755576252937317, Train: 0.5714, Val: 0.2140, Test: 0.2040, Loss2: 1.764222502708435
Epoch: 007, Loss: 1.7288048267364502, Train: 0.4286, Val: 0.2160, Test: 0.2020, Loss2: 1.764222502708435
Epoch: 008, Loss: 1.690131664276123, Train: 0.4286, Val: 0.2080, Test: 0.2020, Loss2: 1.764222502708435
Epoch: 009, Loss: 1.691715955734253, Train: 0.5714, Val: 0.2160, Test: 0.2020, Loss2: 1.764222502708435
Epoch: 010, Loss: 1.6302109956741333, Train: 0.5714, Val: 

In [9]:
optimizer = torch.optim.Adam(gcn.parameters(), lr=0.0005, weight_decay=5e-4)
best_val_acc = final_test_acc = 0
for epoch in range(1, 100):
    loss = train(gcn, data, optimizer, loss='cross_entropy')
    train_acc, val_acc, tmp_test_acc = test(gcn, data)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 0.0025246983859688044, Train: 1.0000, Val: 0.3300, Test: 0.3410
Epoch: 002, Loss: 0.0024562592152506113, Train: 1.0000, Val: 0.3300, Test: 0.3410
Epoch: 003, Loss: 0.002392831025645137, Train: 1.0000, Val: 0.3320, Test: 0.3410
Epoch: 004, Loss: 0.0023339069448411465, Train: 1.0000, Val: 0.3320, Test: 0.3410
Epoch: 005, Loss: 0.002279216656461358, Train: 1.0000, Val: 0.3320, Test: 0.3410
Epoch: 006, Loss: 0.00222847331315279, Train: 1.0000, Val: 0.3320, Test: 0.3410
Epoch: 007, Loss: 0.0021815600339323282, Train: 1.0000, Val: 0.3340, Test: 0.3430
Epoch: 008, Loss: 0.00213805353268981, Train: 1.0000, Val: 0.3380, Test: 0.3440
Epoch: 009, Loss: 0.002097937511280179, Train: 1.0000, Val: 0.3380, Test: 0.3440
Epoch: 010, Loss: 0.002061026869341731, Train: 1.0000, Val: 0.3380, Test: 0.3440
Epoch: 011, Loss: 0.002027220791205764, Train: 1.0000, Val: 0.3380, Test: 0.3440
Epoch: 012, Loss: 0.0019971493165940046, Train: 1.0000, Val: 0.3380, Test: 0.3440
Epoch: 013, Loss: 0.00197

In [22]:
# # take k random training nodes for each class
# data = dataset[0]
# k = 2
# for c in data.y.unique():
#     idx = ((data.y == c) & data.train_mask).nonzero(as_tuple=False).view(-1)
#     idx = idx[torch.randperm(idx.size(0))]
#     idx = idx[k:]
#     data.train_mask[idx] = False

In [36]:
from models import GCNLPA

dataset = cora
data = dataset[0]

k = 2
for c in data.y.unique():
    idx = ((data.y == c) & data.train_mask).nonzero(as_tuple=False).view(-1)
    idx = idx[torch.randperm(idx.size(0))]
    idx = idx[k:]
    data.train_mask[idx] = False

model = GCNLPA(dataset.num_features, 16, dataset.num_classes, dataset.edge_index.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

best_val_acc = final_test_acc = 0
for epoch in range(1, 200):
    op, op_lpa = model(data)
    
    loss1 = F.cross_entropy(op[data.train_mask], data.y[data.train_mask])
    # print(op_alp[train_mask].shape, data.y[train_mask].shape)
    loss2 = F.cross_entropy(op_lpa[data.train_mask], data.y[data.train_mask])
    # print(loss.item(), loss2.item())
    # for p in model.parameters():
    #     print(p)
    loss = loss1 + loss2
    loss.backward()
    optimizer.step()    
    train_acc, val_acc, tmp_test_acc = model.test(data)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    if epoch % 20 == 0:
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc, Loss2 = loss2)
print(f'Best Val Acc: {best_val_acc:.4f}', f'Test Acc: {test_acc:.4f}')

Epoch: 020, Loss: 2.660041332244873, Train: 1.0000, Val: 0.4980, Test: 0.4870, Loss2: 1.290296196937561
Epoch: 040, Loss: 2.4553756713867188, Train: 1.0000, Val: 0.4740, Test: 0.5050, Loss2: 1.2859899997711182
Epoch: 060, Loss: 2.4502530097961426, Train: 1.0000, Val: 0.4740, Test: 0.5050, Loss2: 1.28054678440094
Epoch: 080, Loss: 2.440019130706787, Train: 1.0000, Val: 0.4640, Test: 0.5050, Loss2: 1.274596929550171
Epoch: 100, Loss: 2.4339590072631836, Train: 1.0000, Val: 0.4420, Test: 0.5050, Loss2: 1.2685365676879883
Epoch: 120, Loss: 2.4279847145080566, Train: 1.0000, Val: 0.4380, Test: 0.5050, Loss2: 1.2625625133514404
Epoch: 140, Loss: 2.422208786010742, Train: 1.0000, Val: 0.4300, Test: 0.5050, Loss2: 1.256786584854126
Epoch: 160, Loss: 2.4167556762695312, Train: 1.0000, Val: 0.4240, Test: 0.5050, Loss2: 1.2512913942337036
Epoch: 180, Loss: 2.411555290222168, Train: 1.0000, Val: 0.4200, Test: 0.5050, Loss2: 1.246132731437683
Best Val Acc: 0.5280 Test Acc: 0.5050


In [ ]:
# outs.shape, outs[0, 0, :].sum(), (outs[0, 0, :] == outs[2, 0, :]).sum()

In [ ]:
best_val_acc = final_test_acc = 0

model.train()
for epoch in range(1, 100):
    optimizer.zero_grad()
    loss = model.loss()
    loss.backward()
    optimizer.step()
    
    train_acc, val_acc, tmp_test_acc = model.test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)